In [1]:
# Importing Libraries
import pyodbc
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt


In [2]:
# SQL Database Connection
cnxn_str = ("Driver={SQL Server Native Client 11.0};"
            "Server=LAPTOP-T8RJOBVC\CFSQL;"
            "Database=Optimax;"
            "Trusted_Connection=yes;")
cnxn = pyodbc.connect(cnxn_str)
cursor = cnxn.cursor()


##### Requirement 1: view the most selling drug by store

In [3]:
# Medicine Table
medicines = pd.read_sql_query (''' select * from medicines''', cnxn)
medicines_df = pd.DataFrame(medicines)
medicines_df


C:\Users\chinnu\AppData\Local\Temp\ipykernel_11540\144092283.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  medicines = pd.read_sql_query (''' select * from medicines''', cnxn)


,medicine_id,medicine_name,expiry_date,manufactured_date,medicine_type,key_ingredient,standard_price,role_id
0,1,Combiflam,2025-11-10,2021-11-03,analgesic,Ibuprofen,5.0,1
1,2,Combiflam,2025-12-10,2021-05-03,analgesic,Ibuprofen,5.0,2


In [4]:
# Medicine vendor table
medicine_vendor = pd.read_sql_query (''' select * from med_store''', cnxn)
medicine_vendor = pd.DataFrame(medicine_vendor)
medicine_vendor

C:\Users\chinnu\AppData\Local\Temp\ipykernel_11540\1353358312.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  medicine_vendor = pd.read_sql_query (''' select * from med_store''', cnxn)


,medicine_id,store_id,stock_in_hand,transit_quantity,role_id
0,1,9991,10,2,1


In [5]:
medicine_store = medicines_df.merge(medicine_vendor, on = ['medicine_id','role_id'], how = 'left')
medicine_store

,medicine_id,medicine_name,expiry_date,manufactured_date,medicine_type,key_ingredient,standard_price,role_id,store_id,stock_in_hand,transit_quantity
0,1,Combiflam,2025-11-10,2021-11-03,analgesic,Ibuprofen,5.0,1,9991.0,10.0,2.0
1,2,Combiflam,2025-12-10,2021-05-03,analgesic,Ibuprofen,5.0,2,NaN,NaN,NaN


In [6]:
# Viewing the most selling drug
med_count = medicine_store.groupby(['medicine_name', 'store_id']).agg({'medicine_name' : 'count'}).rename(columns={'medicine_name':'medicine_count'}).reset_index()

# For each store the max selling drug
med_count_max_by_Store = med_count.groupby(['medicine_name', 'store_id']).max().reset_index()
med_count_max_by_Store

,medicine_name,store_id,medicine_count
0,Combiflam,9991.0,1


In [12]:

# Display it as a bar plot
# x axis has store name and y axis the medicine count
fig = px.bar(med_count_max_by_Store, x="store_id", y="medicine_count", text="medicine_name", title="Most Selling Drug", 
color_discrete_sequence =["#93D500"])
fig.update_xaxes(type='category')
fig.show()